# Classifiez automatiquement des biens de consommation

- **Projet 6 du parcours « Data Scientist » d’OpenClassrooms**
- **Mark Creasey**

## Partie 1 : Classification de textes


<!--TABLE OF CONTENTS-->

- [1. Compréhension du problème](#1.-Compréhension-du-problème)
  - [1.1 Mission](#1.1-Mission)
  - [1.2 Requirements : Bibliothèques utilisées dans ce notebook](#1.2-Requirements-:-Bibliothèques-utilisées-dans-ce-notebook)
  - [1.3 Des fonctions utilitaires](#1.3-Des-fonctions-utilitaires)
  - [1.4 Des routines statistiques](#1.4-Des-routines-statistiques)
- [2. Import et nettoyage des données](#2.-Import-et-nettoyage-des-données)


# 1. Compréhension du problème

## 1.1 Mission

## Mission

Etudier la faisabilité d'un **moteur de classification** des articles en différentes catégories,
avec un niveau de précision suffisant.

- **réaliser une première étude de faisabilité d'un moteur de classification**, d'articles, basé sur
  une image et une description, pour l'automatisation de l'attribution de la catégorie de l'article.

- **analyser le jeu de données** en **réalisant un prétraitement** des descriptions des produits et
  des images, une **réduction de dimension**, puis un **clustering**, à présenter sous la forme de
  graphiques en deux dimensions, et confirmés par un calcul de similarité entre les catégories
  réelles et les clusters.
- illustre que les caractéristiques extraites permettent de regrouper des produits de même
  catégorie.

- démontrer, par cette approche de modélisation, la faisabilité de regrouper automatiquement des
  produits de même catégorie


## 1.2 Requirements : Bibliothèques utilisées dans ce notebook

- voir [`requirements.txt`](./requirements.txt) pour les versions des bibliothèques testées avec ce notebook


In [1]:
# si ce notebook ne marche pas avec les versions
# des bibliothèques dans votre environnement, alors
# decommentarise la ligne suivant pour des versions testées:

# %pip install -r requirements.txt


In [2]:
def install_libraries(required={}) -> None:
    """
    Installation des bibliothèques manquantes
    https://stackoverflow.com/questions/44210656/
    """
    import sys
    import subprocess
    import pkg_resources
    installed = {pkg.key for pkg in pkg_resources.working_set}
    missing = set(required) - set(installed)
    if missing:
        print(f'missing libraries: {missing}')
        python = sys.executable
        subprocess.check_call([python, '-m', 'pip', 'install', *missing],
                              stdout=subprocess.DEVNULL)


required_libraries = {'numpy', 'pandas',
                      'matplotlib', 'seaborn', 'scikit-learn',
                      'nltk'}
install_libraries(required_libraries)


### 1.2.1 Import des bibliothèques


In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import nltk


### 1.2.2 Liste des versions des bibliothèques utilisées


In [4]:
from platform import python_version

python_version()
print('versions des bibliothèques utilisées:')
print('; '.join(f'{m.__name__}=={m.__version__}' for m in globals(
).values() if getattr(m, '__version__', None)))


versions des bibliothèques utilisées:
numpy==1.21.5; pandas==1.1.5; seaborn==0.11.2; sklearn==1.0.2; nltk==3.7


### 1.2.3 Configuration défauts d'affichage


In [5]:
pd.set_option('display.max_columns', 200)  # pour afficher toutes les colonnes
pd.set_option('display.max_rows', 20)  # pour afficher max 10 lignes
pd.set_option('display.max_colwidth', 100)  # pour afficher toutes la text
pd.set_option('display.precision', 2)
pd.options.display.float_format = '{:.2f}'.format  # currency, lat,lng

%matplotlib inline
sns.set_theme(style="white", context="notebook")
sns.set_color_codes("pastel")
sns.set_palette("tab20")


## 1.3 Des fonctions utilitaires

### 1.3.1 Enregistrement des graphiques

Pour enregistrer les graphiques, define **`SAVE_IMAGES = True`**


In [6]:
SAVE_IMAGES = True
IMAGE_FOLDER = './images/analyse'
if not os.path.exists(IMAGE_FOLDER):
    os.makedirs(IMAGE_FOLDER)


In [7]:
def to_png(fig_name=None) -> None:
    """
    Register the current plot figure as an image in a file.
    Must call plt.show() or show image (by calling to_png() as last row in python cell)
    to apply the call 'bbox_inches=tight', to be sure to include the whole title / legend
    in the plot area.
    """

    def get_title() -> str:
        """find current plot title (or suptitle if more than one plot)"""
        if plt.gcf()._suptitle is None:  # noqa
            return plt.gca().get_title()
        else:
            return plt.gcf()._suptitle.get_text()  # noqa

    if SAVE_IMAGES:
        if fig_name is None:
            fig_name = get_title()
        elif len(fig_name) < 9:
            fig_name = f'{fig_name}_{get_title()}'
        fig_name = fig_name.replace(' ', '_').replace(':', '-').replace(
            '.', '-').replace('/', '_').replace('>', 'gt.').replace('<', 'lt.')
        print(f'"{fig_name}.png"')
        plt.gcf().savefig(
            f'{IMAGE_FOLDER}/{fig_name}.png', bbox_inches='tight')


### 1.3.2 Vérifier que les colonnes sont dans le dataframe

- sans changer l'ordre des colonnes


In [8]:
def cols_in_df(df: pd.DataFrame, colonnes: list = None) -> list:
    """
    Procedure pour retourner les colonnes existantes dans le dataframe dans la même ordre.
    Utiliser pour assurer que les colonnes existe.
    """
    ret_cols = []
    for col in colonnes:
        if col in df.columns:
            ret_cols.append(col)
    return ret_cols


# 2. Import et nettoyage des données

## Les données

Un premier jeu de données d’articles avec la photo et une description associée :
[le lien pour télécharger](https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/Parcours_data_scientist/Projet+-+Textimage+DAS+V2/Dataset+projet+pre%CC%81traitement+textes+images.zip)


## Import des données

Le données du fichier zip (329 Mo) sont éxtraites dans le dossier définit par DATA_FOLDER ci-dessous.


In [9]:
ENV = 'local'

if ENV == 'local':
    # local development
    DATA_FOLDER = 'data/raw'
    OUT_FOLDER = 'data/out'
    IMAGE_FOLDER = 'images/textes'

if ENV == 'colab':
    # Colaboratory - uncomment les 2 lignes suivant pour connecter à votre drive
    from google.colab import drive
    drive.mount('/content/drive')
    DATA_FOLDER = '/content/drive/MyDrive/data/OC6'
    OUT_FOLDER = '/content/drive/MyDrive/data/OC6'
    IMAGE_FOLDER = '/content/drive/MyDrive/images/OC6/nettoyage'


In [10]:
# Données (DATA_FOLDER)
ZIPPED_DATA_FILENAME = f'Dataset+projet+prétraitement+textes+images.zip'
RAW_DATA_FILENAME = 'Flipkart/flipkart_com-ecommerce_sample_1050.csv'


def os_make_dir(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)


def os_path_join(folder, file):
    """remplacement pour `os.path.join(folder, file)` sur windows"""
    return f'{folder}/{file}'


os_make_dir(IMAGE_FOLDER)
RAW_DATA = os_path_join(DATA_FOLDER, RAW_DATA_FILENAME)
DATA_ZIPPED = os_path_join(DATA_FOLDER, ZIPPED_DATA_FILENAME)
print(f'data file: {RAW_DATA}')


data file: data/raw/Flipkart/flipkart_com-ecommerce_sample_1050.csv


In [11]:
def unzip_data_si_besoin(env=ENV):
    """
    procedure pour unzip sur Google Drive via Google Colab
    """
    if os.path.exists(RAW_DATA):
        print(f'data CSV file exists ({RAW_DATA})')
    else:
        print(f'data CSV file does not exist ({RAW_DATA})')
        if env == 'colab' and os.path.exists(DATA_ZIPPED):
            # uncomment les 3 lignes suivants
            print(f'unzipping {DATA_ZIPPED}')
            !unzip {DATA_ZIPPED} - d {DATA_FOLDER}
            print(f'{DATA_ZIPPED} has been unzipped')
            if os.path.exists(RAW_DATA):
                print(f'data CSV file now exists ({RAW_DATA})')
        else:
            print(f'zipped data does not exist ({DATA_ZIPPED})')


unzip_data_si_besoin(ENV)


data CSV file exists (data/raw/Flipkart/flipkart_com-ecommerce_sample_1050.csv)


In [12]:
def read_file_rows(file_path=RAW_DATA, nb_rows=1):
    fp = open(file_path, encoding='UTF-8')
    for i in range(nb_rows + 1):
        row = fp.readline()
        print(f'Line {i} :\n{row}')


read_file_rows(RAW_DATA, nb_rows=1)


Line 0 :
uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications

Line 1 :
55b85ea15a1536d46b7190ad6fff8ce7,2016-04-30 03:22:56 +0000,http://www.flipkart.com/elegance-polyester-multicolor-abstract-eyelet-door-curtain/p/itmeg7bkzdgthcts?pid=CRNEG7BKMFFYHQ8Z,Elegance Polyester Multicolor Abstract Eyelet Door Curtain,"[""Home Furnishing >> Curtains & Accessories >> Curtains >> Elegance Polyester Multicolor Abstract Eyelet Do...""]",CRNEG7BKMFFYHQ8Z,1899.0,899.0,55b85ea15a1536d46b7190ad6fff8ce7.jpg,False,"Key Features of Elegance Polyester Multicolor Abstract Eyelet Door Curtain Floral Curtain,Elegance Polyester Multicolor Abstract Eyelet Door Curtain (213 cm in Height, Pack of 2) Price: Rs. 899 This curtain enhances the look of the interiors.This curtain is made from 100% high quality polyester fabric.It features an eyelet style stitch with 

#### Format des données :

- La premiere ligne contient les en-têtes (column names)
- Les données semblent séparées par virgule
- On utilise `encoding = UTF-8`
- La colonne `product_specifications` est composé de key values.


In [13]:
df_data = pd.read_csv(RAW_DATA, sep=',', header=0, encoding='UTF-8')
df_data.head(2)


,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,55b85ea15a1536d46b7190ad6fff8ce7,2016-04-30 03:22:56 +0000,http://www.flipkart.com/elegance-polyester-multicolor-abstract-eyelet-door-curtain/p/itmeg7bkzdg...,Elegance Polyester Multicolor Abstract Eyelet Door Curtain,"[""Home Furnishing >> Curtains & Accessories >> Curtains >> Elegance Polyester Multicolor Abstrac...",CRNEG7BKMFFYHQ8Z,1899.00,899.00,55b85ea15a1536d46b7190ad6fff8ce7.jpg,False,"Key Features of Elegance Polyester Multicolor Abstract Eyelet Door Curtain Floral Curtain,Elegan...",No rating available,No rating available,Elegance,"{""product_specification""=>[{""key""=>""Brand"", ""value""=>""Elegance""}, {""key""=>""Designed For"", ""value..."
1,7b72c92c2f6c40268628ec5f14c6d590,2016-04-30 03:22:56 +0000,http://www.flipkart.com/sathiyas-cotton-bath-towel/p/itmegfzhxbyucwgn?pid=BTWEGFZHGBXPHZUH,Sathiyas Cotton Bath Towel,"[""Baby Care >> Baby Bath & Skin >> Baby Bath Towels >> Sathiyas Baby Bath Towels >> Sathiyas Cot...",BTWEGFZHGBXPHZUH,600.00,449.00,7b72c92c2f6c40268628ec5f14c6d590.jpg,False,"Specifications of Sathiyas Cotton Bath Towel (3 Bath Towel, Red, Yellow, Blue) Bath Towel Featur...",No rating available,No rating available,Sathiyas,"{""product_specification""=>[{""key""=>""Machine Washable"", ""value""=>""Yes""}, {""key""=>""Material"", ""val..."


### Nettoyage


In [14]:
df_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uniq_id                  1050 non-null   object 
 1   crawl_timestamp          1050 non-null   object 
 2   product_url              1050 non-null   object 
 3   product_name             1050 non-null   object 
 4   product_category_tree    1050 non-null   object 
 5   pid                      1050 non-null   object 
 6   retail_price             1049 non-null   float64
 7   discounted_price         1049 non-null   float64
 8   image                    1050 non-null   object 
 9   is_FK_Advantage_product  1050 non-null   bool   
 10  description              1050 non-null   object 
 11  product_rating           1050 non-null   object 
 12  overall_rating           1050 non-null   object 
 13  brand                    712 non-null    object 
 14  product_specifications  

Il y a peu de données manquantes:

- 1 prix non-indiqué
- 75% des données n'ont pas de brand


### Catégories

L'objectif est de pouvoir classifier les produits dans le `product_category_tree` de façon automatique.


In [15]:
print(df_data['product_category_tree'].nunique())
print(df_data['product_category_tree'].str.lower().nunique())


642
639


Avec 1000 echantillons, il sera dificil d'automatiser la classification en 640 catégories.

On regard les niveaux plus haut de l'arbre.


In [16]:
def get_category_count(row):
    """obtenir le nombre de catégories pour un produit"""
    return row.count('[')


df_data['product_category_tree'].map(lambda x: get_category_count(x)).max()


1

Chaque produit est associe à seulement une categorie dans le category tree

La colonne `product_category_tree` est un arbre, separé par '>>'.

Quel est son profondeur ?


In [17]:
def get_depth(categorie: str) -> int:
    """obtenir le nombre de niveaux de catégories"""
    return len(categorie.split(' >> '))


df_data['product_category_tree'].map(lambda x: get_depth(x)).max()


7

Les produits ont jusqu'à 7 niveaux de catégorie


In [18]:

df_data['product_category_tree'].str.lstrip('["').str.split(' >>').head()


0    [Home Furnishing,  Curtains & Accessories,  Curtains,  Elegance Polyester Multicolor Abstract Ey...
1    [Baby Care,  Baby Bath & Skin,  Baby Bath Towels,  Sathiyas Baby Bath Towels,  Sathiyas Cotton B...
2    [Baby Care,  Baby Bath & Skin,  Baby Bath Towels,  Eurospa Baby Bath Towels,  Eurospa Cotton Ter...
3    [Home Furnishing,  Bed Linen,  Bedsheets,  SANTOSH ROYAL FASHION Bedsheets,  SANTOSH ROYAL FASHI...
4    [Home Furnishing,  Bed Linen,  Bedsheets,  Jaipur Print Bedsheets,  Jaipur Print Cotton Floral K...
Name: product_category_tree, dtype: object

In [19]:
def remove_brackets(node:str):
    node=node.replace('["','').replace('"]','')
    return node

def create_categ_level(df):
    df_cat = (df['product_category_tree']
                .map(lambda x:remove_brackets(x))
                .str.split(' >> ', expand=True))
    print(df_cat.shape)
    depth= len(df_cat.columns)
    df_cat.columns=[f'categ_level_{i}' for i in range(1,depth+1)]
    return pd.concat([df,df_cat], axis=1)
    # return df_cat

df_cleaned = df_data.pipe(create_categ_level)
    

(1050, 7)


### Categorie niveau 1


In [20]:
df_cleaned['categ_level_1'].value_counts()


Home Decor & Festive Needs    150
Beauty and Personal Care      150
Baby Care                     150
Home Furnishing               150
Watches                       150
Computers                     150
Kitchen & Dining              150
Name: categ_level_1, dtype: int64

On peux essayer de classifier par niveau 1


### Catégories - niveau 2


In [21]:
print(df_cleaned['categ_level_2'].nunique())
pd.concat([df_cleaned['categ_level_2'].value_counts().head(20),
           df_cleaned['categ_level_2'].value_counts(
).cumsum().head(20).rename('cum_sum'),
], axis=1)


62


,categ_level_2,cum_sum
Wrist Watches,149,149
Laptop Accessories,87,236
Infant Wear,84,320
Coffee Mugs,74,394
Showpieces,71,465
Fragrances,65,530
Bed Linen,65,595
Network Components,49,644
Cookware,27,671
Table Decor & Handicrafts,27,698


Pour l'arbre de categories, 90% des produits sont dans les 20 plus grands catégories de niveau 2.

- On peut evaluer la performance de classification à niveau 2


## Préparation des déscriptions des données

L'objectif est d'utiliser les déscriptions des données pour classifier dans les catégories de niveaux 1, 2, 3...

Il faut préparer les déscriptions:

- mettre tout en minuscule
- retire les mots de liaison,
- retirer la ponctuation

On utilise la librairie nltk (Natural Language Toolkit), pour aider la nettoyage


### Nettoyage du texte

#### Import des librairies


In [22]:
import nltk

# stopwords
nltk.download('stopwords')
# punctuation
nltk.download('punkt')
# lemmatization
nltk.download('wordnet')
# Open Multilingual Wordnet Project lemmatizations
nltk.download('omw-1.4')
pass


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mc\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Tokenization


In [23]:
# Tokenizer
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize


def tokenizer_fct(sentence):
    # print(sentence)
    sentence_clean = (sentence
                      .replace('-', ' ')
                      .replace('+', ' ')
                      .replace('/', ' ')
                      .replace('.', '. ')
                      .replace('#', ' '))
    word_tokens = word_tokenize(sentence_clean)
    return word_tokens


print(tokenizer_fct(df_data['description'][6]))


['Camerii', 'WM64', 'Elegance', 'Analog', 'Watch', 'For', 'Men', ',', 'Boys', 'Buy', 'Camerii', 'WM64', 'Elegance', 'Analog', 'Watch', 'For', 'Men', ',', 'Boys', 'WM64', 'Online', 'at', 'Rs', '.', '449', 'in', 'India', 'Only', 'at', 'Flipkart', '.', 'com', '.', 'Great', 'Discounts', ',', 'Only', 'Genuine', 'Products', ',', '30', 'Day', 'Replacement', 'Guarantee', ',', 'Free', 'Shipping', '.', 'Cash', 'On', 'Delivery', '!']


### Stop words


In [24]:
from typing import List
from nltk.corpus import stopwords

stop_w = list(set(stopwords.words('english'))) + \
    ['[', ']', ',', '.', ':', '?', '(', ')']


def stop_word_filter_fct(list_words: List[str]) -> List[str]:
    """Stop words"""
    filtered_w = [w for w in list_words if not w in stop_w]
    filtered_w2 = [w for w in filtered_w if len(w) > 2]
    return filtered_w2


print(stop_word_filter_fct(tokenizer_fct(df_data['description'][6])))


['Camerii', 'WM64', 'Elegance', 'Analog', 'Watch', 'For', 'Men', 'Boys', 'Buy', 'Camerii', 'WM64', 'Elegance', 'Analog', 'Watch', 'For', 'Men', 'Boys', 'WM64', 'Online', '449', 'India', 'Only', 'Flipkart', 'com', 'Great', 'Discounts', 'Only', 'Genuine', 'Products', 'Day', 'Replacement', 'Guarantee', 'Free', 'Shipping', 'Cash', 'Delivery']


In [25]:
print(stop_word_filter_fct(tokenizer_fct(df_data['description'][6])))


['Camerii', 'WM64', 'Elegance', 'Analog', 'Watch', 'For', 'Men', 'Boys', 'Buy', 'Camerii', 'WM64', 'Elegance', 'Analog', 'Watch', 'For', 'Men', 'Boys', 'WM64', 'Online', '449', 'India', 'Only', 'Flipkart', 'com', 'Great', 'Discounts', 'Only', 'Genuine', 'Products', 'Day', 'Replacement', 'Guarantee', 'Free', 'Shipping', 'Cash', 'Delivery']


In [26]:
from typing import List


def lower_start_fct(list_words: List[str]) -> List[str]:
    """lower case et alpha"""
    lw = [w.lower() for w in list_words if (not w.startswith("@"))
          #                                   and (not w.startswith("#"))
          and (not w.startswith("http"))]
    return lw


print(lower_start_fct(stop_word_filter_fct(
    tokenizer_fct(df_data['description'][6]))))


['camerii', 'wm64', 'elegance', 'analog', 'watch', 'for', 'men', 'boys', 'buy', 'camerii', 'wm64', 'elegance', 'analog', 'watch', 'for', 'men', 'boys', 'wm64', 'online', '449', 'india', 'only', 'flipkart', 'com', 'great', 'discounts', 'only', 'genuine', 'products', 'day', 'replacement', 'guarantee', 'free', 'shipping', 'cash', 'delivery']


In [27]:
from typing import List
from nltk.stem import WordNetLemmatizer


def lemma_fct(list_words: List[str]) -> List[str]:
    """
    Lemmatizer (base d'un mot)
    """
    lemmatizer = WordNetLemmatizer()
    lem_w = [lemmatizer.lemmatize(w) for w in list_words]
    return lem_w


In [28]:
# test de lemmatizer
print(lemma_fct(lower_start_fct(
    stop_word_filter_fct(tokenizer_fct(df_data['description'][6])))))


['camerii', 'wm64', 'elegance', 'analog', 'watch', 'for', 'men', 'boy', 'buy', 'camerii', 'wm64', 'elegance', 'analog', 'watch', 'for', 'men', 'boy', 'wm64', 'online', '449', 'india', 'only', 'flipkart', 'com', 'great', 'discount', 'only', 'genuine', 'product', 'day', 'replacement', 'guarantee', 'free', 'shipping', 'cash', 'delivery']


# Préparation commune des traitements


In [29]:
from nltk.tokenize import sent_tokenize, word_tokenize



def transform_bow_fct(desc_text):
    """
    Fonction de préparation du texte pour le bag of words
    (Countvectorizer et Tf_idf, Word2Vec)
    """
    word_tokens = tokenizer_fct(desc_text)
    sw = stop_word_filter_fct(word_tokens)
    lw = lower_start_fct(sw)
    # lem_w = lemma_fct(lw)
    transf_desc_text = ' '.join(lw)
    return transf_desc_text


def transform_bow_lem_fct(desc_text):
    """
    Fonction de préparation du texte pour le bag of words 
    avec lemmatization
    """
    word_tokens = tokenizer_fct(desc_text)
    sw = stop_word_filter_fct(word_tokens)
    lw = lower_start_fct(sw)
    lem_w = lemma_fct(lw)
    transf_desc_text = ' '.join(lem_w)
    return transf_desc_text


def transform_dl_fct(desc_text):
    """
    Fonction de préparation du texte pour le Deep learning 
    (USE et BERT)
    """
    word_tokens = tokenizer_fct(desc_text)
#    sw = stop_word_filter_fct(word_tokens)
    lw = lower_start_fct(word_tokens)
    # lem_w = lemma_fct(lw)
    transf_desc_text = ' '.join(lw)
    return transf_desc_text


In [30]:

def transform_field(df: pd.DataFrame, text_col: str = 'description'):
    df = df.copy()
    df['sentence_bow'] = df[text_col].apply(lambda x: transform_bow_fct(x))
    df['sentence_bow_lem'] = df[text_col].apply(
        lambda x: transform_bow_lem_fct(x))
    df['sentence_dl'] = df[text_col].apply(lambda x: transform_dl_fct(x))
    print(df.shape)
    return df


df_cleaned.pipe(transform_field).head(1)


(1050, 25)


,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,categ_level_1,categ_level_2,categ_level_3,categ_level_4,categ_level_5,categ_level_6,categ_level_7,sentence_bow,sentence_bow_lem,sentence_dl
0,55b85ea15a1536d46b7190ad6fff8ce7,2016-04-30 03:22:56 +0000,http://www.flipkart.com/elegance-polyester-multicolor-abstract-eyelet-door-curtain/p/itmeg7bkzdg...,Elegance Polyester Multicolor Abstract Eyelet Door Curtain,"[""Home Furnishing >> Curtains & Accessories >> Curtains >> Elegance Polyester Multicolor Abstrac...",CRNEG7BKMFFYHQ8Z,1899.00,899.00,55b85ea15a1536d46b7190ad6fff8ce7.jpg,False,"Key Features of Elegance Polyester Multicolor Abstract Eyelet Door Curtain Floral Curtain,Elegan...",No rating available,No rating available,Elegance,"{""product_specification""=>[{""key""=>""Brand"", ""value""=>""Elegance""}, {""key""=>""Designed For"", ""value...",Home Furnishing,Curtains & Accessories,Curtains,Elegance Polyester Multicolor Abstract Eyelet Do...,None,None,None,key features elegance polyester multicolor abstract eyelet door curtain floral curtain elegance ...,key feature elegance polyester multicolor abstract eyelet door curtain floral curtain elegance p...,"key features of elegance polyester multicolor abstract eyelet door curtain floral curtain , eleg..."


In [31]:

def calc_length_bow(df: pd.DataFrame):
    df = df.copy()
    df['length_bow'] = df['sentence_bow'].apply(
        lambda x: len(word_tokenize(x)))
    print(f"max length bow : {df['length_bow'].max()}")
    df['length_dl'] = df['sentence_dl'].apply(lambda x: len(word_tokenize(x)))
    print(f"max length dl : {df['length_dl'].max()}")
    return df


(df_cleaned
 .pipe(transform_field)
 .pipe(calc_length_bow)
 .head(1))


(1050, 25)
max length bow : 377
max length dl : 636


,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,categ_level_1,categ_level_2,categ_level_3,categ_level_4,categ_level_5,categ_level_6,categ_level_7,sentence_bow,sentence_bow_lem,sentence_dl,length_bow,length_dl
0,55b85ea15a1536d46b7190ad6fff8ce7,2016-04-30 03:22:56 +0000,http://www.flipkart.com/elegance-polyester-multicolor-abstract-eyelet-door-curtain/p/itmeg7bkzdg...,Elegance Polyester Multicolor Abstract Eyelet Door Curtain,"[""Home Furnishing >> Curtains & Accessories >> Curtains >> Elegance Polyester Multicolor Abstrac...",CRNEG7BKMFFYHQ8Z,1899.00,899.00,55b85ea15a1536d46b7190ad6fff8ce7.jpg,False,"Key Features of Elegance Polyester Multicolor Abstract Eyelet Door Curtain Floral Curtain,Elegan...",No rating available,No rating available,Elegance,"{""product_specification""=>[{""key""=>""Brand"", ""value""=>""Elegance""}, {""key""=>""Designed For"", ""value...",Home Furnishing,Curtains & Accessories,Curtains,Elegance Polyester Multicolor Abstract Eyelet Do...,None,None,None,key features elegance polyester multicolor abstract eyelet door curtain floral curtain elegance ...,key feature elegance polyester multicolor abstract eyelet door curtain floral curtain elegance p...,"key features of elegance polyester multicolor abstract eyelet door curtain floral curtain , eleg...",158,258
